# Basic Data Cleaning
This portion of the notebook deals with initial exploration of the data for cleaning purposes.

There are three input json files.

The schema for each file is as follows (copied verbatim from provided materials):

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


Note: Users that didn't provide their age have their age encoded as 118.
Note: BOGO stands for "buy one get one"

In [1]:
import pandas as pd
import numpy as np
# altair for plotting
import altair as alt

# Sometimes I use display, but I'm not very consistent in doing so
from IPython.display import display

%matplotlib inline

In [2]:
# Read in data files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [3]:
print(f"Portfolio shape: {portfolio.shape}")
print(f"Profile shape: {profile.shape}")
print(f"Transcript shape: {transcript.shape}")

Portfolio shape: (10, 6)
Profile shape: (17000, 5)
Transcript shape: (306534, 4)


In [4]:
# Heads of each dataframe
display(portfolio.head(1))
display(profile.head(1))
display(transcript.head(1))

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0


In [5]:
# Summary information
print("PORTFOLIO \n")
portfolio.info()
print('\n===============================================\n')
print("PROFILE \n")
profile.info()
print('\n===============================================\n')
print("TRANSCRIPT \n")
transcript.info()

PORTFOLIO 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


PROFILE 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB

The profile dataframe does contain nulls. Note age also has nulls encoded as 118.

## Cleaning the `portfolio` data

In [6]:
# We can display the whole 10 lines
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [7]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


I'll create a function to do the following cleaning steps:
- Convert the channels column to individual channels of 0's and 1's
- Reorder the columns
- Rename the id column to "offer_id" so we can combine it with the other dataframes

In [8]:
def clean_portfolio(df, channel_types=['web', 'email', 'mobile', 'social']):
    """
    Cleans the raw portfolio dataframe through various cleaning steps.
    
    Input:
    df - the raw portfolio dataframe from portfolio.json
    channel_types - a list of str's for the different media channels
    
    Output:
    clean_df - the cleaned dataframe.
    
    Cleaning steps:
    - Takes channels column and converts it into 0/1's columns for each channel.
    - Reorder the columns.
    - Rename the id column to "offer_id".    
    """
    # Get a dataframe for channel data
    # Iterates through each list in the .channels series
    # to pull out whether each channel is present or not.
    campaigns = []
    for campaign in df.channels:
        campaign = set(campaign)
        channels = []
        for channel in channel_types:
            if channel in campaign:
                channels.append(1)
            else:
                channels.append(0)
        campaigns.append(channels)

    channel_frame = pd.DataFrame(campaigns)
    channel_frame.columns = channel_types
    
    # Reorder the original dataframe as well replacing the channels column
    # with the new channel dataframe
    clean_df = pd.concat([df[['id']],
                          channel_frame,
                          df[['offer_type', 'duration', 'difficulty', 'reward']]],
                          axis=1)
    
    # Rename id as campaign_id
    clean_df = clean_df.rename(columns={'id':'offer_id'})
                          
    return clean_df

In [9]:
portfolio = clean_portfolio(portfolio)

In [10]:
portfolio.head()

,offer_id,web,email,mobile,social,offer_type,duration,difficulty,reward
0,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,bogo,7,10,10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,bogo,5,10,10
2,3f207df678b143eea3cee63160fa8bed,1,1,1,0,informational,4,0,0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,bogo,7,5,5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,discount,10,20,5


## Cleaning the `profile` data

In [11]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [12]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [13]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [14]:
# What fraction of age, gender, and income is NA (not provided by user)?
# Note: Age NA's are encoded as 118.
print(f"Gender NA fraction: {round(profile.gender.isna().mean(), 4)}")
print(f"Age NA fraction: {round((profile.age == 118).mean(), 4)}")
print(f"Income NA fraction: {round(profile.income.isna().mean(), 4)}")

Gender NA fraction: 0.1279
Age NA fraction: 0.1279
Income NA fraction: 0.1279


In [15]:
# Given that all the the NA fractions are the same,
# are NAs all on the same entries?
print("Are all the NAs in gender, age, and income on the same rows?")
print(
    (profile.gender.isna() == (profile.age == 118)).all() == \
    (profile.gender.isna() == profile.income.isna()).all()
)

Are all the NAs in gender, age, and income on the same rows?
True


In [16]:
# There are no NAs in became_member_on and id columns
print(f"id NA fraction: {profile.id.isna().mean()}")
print(f"became_member_on NA fraction: {profile.became_member_on.isna().mean()}")

id NA fraction: 0.0
became_member_on NA fraction: 0.0


I'll define a function to do the following steps in cleaning the profile data:

- Standardize the different types of NAs (None, age == 118) into np.nan.
- Convert 'became_member_on' into datetime objects.
- Reorder the columns.
- Rename the user id.


In [17]:
def clean_profile(df):
    """
    Cleans the raw portfolio dataframe through various cleaning steps.
    
    Input:
    df - the raw profile dataframe from profile.json
    
    Output:
    clean_df - the cleaned dataframe.
    
    Cleaning steps:
    - Standardize the different types of NAs (None, age == 118) into np.nan.
    - Convert 'became_member_on' into datetime objects.
    - Reorder the columns.
    - Rename the user id.
    """
    df = df.copy()
    # Standardize NAs to numpy nan
    df['age'] = df['age'].replace(118, np.nan)
    df['gender'] = df['gender'].fillna(np.nan)
    # Convert str to datetime
    df['became_member_on'] = pd.to_datetime(df['became_member_on'], format='%Y%m%d')    
    # Reorder the columns
    clean_df = df[['id', 'gender', 'age', 'income', 'became_member_on']]
    # Relabel id as user_id
    clean_df = clean_df.rename(columns={'id':'customer_id'})
    
    return clean_df
    

In [18]:
profile = clean_profile(profile)
profile.head()

,customer_id,gender,age,income,became_member_on
0,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,NaN,2017-02-12
1,0610b486422d4921ae7d2bf64640c50b,F,55.0,112000.0,2017-07-15
2,38fe809add3b4fcf9315a9694bb96ff5,NaN,NaN,NaN,2018-07-12
3,78afa995795e4d85b5d9ceeca43f5fef,F,75.0,100000.0,2017-05-09
4,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,NaN,2017-08-04


## Cleaning the `transcript` data



In [19]:
transcript.shape

(306534, 4)

In [20]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [21]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [22]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [23]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [24]:
transcript[transcript.event == 'offer completed'].head()

,person,event,value,time
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,0
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,0
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0
12692,676506bad68e4161b9bbaffeb039626b,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0...,0


The value column holds dictionaries with different values depending on what the event column is.

I'll write a function to clean data by:

- Replace spaces in event strings to underscores.
- Expanding out "value" column.


In [25]:
def clean_transcript(df):
    """
    Cleans the raw transcript dataframe through various cleaning steps.
    
    Input:
    df - the raw transcript dataframe from transcript.json
    
    Output:
    clean_df - the cleaned dataframe.
    
    Cleaning steps:
    - Replace spaces in event strings to underscores.
    - Expand out "value" column.
    """
    df = df.copy()
    # Replace space with underscore in event column
    df['event'] = df['event'].str.replace(' ', '_')
    
    # Expand the value column into multiple columns
    value_frame = pd.DataFrame(df['value'].to_list())
    # Combine 'offer id' and 'offer_id' columns
    # First get the non nulls from each and check that they don't overlap
    a = value_frame['offer id'].notnull()
    b = value_frame['offer_id'].notnull()
    assert ~((a & b).any()) # True if no overlap
    # Make the combined column and drop 'offer id'
    value_frame['offer_id'] = value_frame['offer_id'].combine_first(value_frame['offer id'])
    value_frame = value_frame.drop(labels=['offer id'], axis=1)
    
    # Concatenate and reorder the columns
    clean_df = pd.concat([df, value_frame], axis=1)
    clean_df = clean_df[['person', 'time', 'event', 'amount', 'reward', 'offer_id']]
    
    # Rename the 'person column'
    clean_df = clean_df.rename(columns={'person':'customer_id'})
    
    return clean_df
    

In [26]:
transcript = clean_transcript(transcript)

In [27]:
transcript.head()

,customer_id,time,event,amount,reward,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,offer_received,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,offer_received,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,offer_received,NaN,NaN,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,offer_received,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,offer_received,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


## Reconciliating IDs
There's two sets of ids: one belonging to each customer, and one belonging to each offer from a campaign. These are hashes, and it'd be easier to work with if they were simply numbers.

In [28]:
portfolio.head()

,offer_id,web,email,mobile,social,offer_type,duration,difficulty,reward
0,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,bogo,7,10,10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,bogo,5,10,10
2,3f207df678b143eea3cee63160fa8bed,1,1,1,0,informational,4,0,0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,bogo,7,5,5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,discount,10,20,5


In [29]:
profile.head()

,customer_id,gender,age,income,became_member_on
0,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,NaN,2017-02-12
1,0610b486422d4921ae7d2bf64640c50b,F,55.0,112000.0,2017-07-15
2,38fe809add3b4fcf9315a9694bb96ff5,NaN,NaN,NaN,2018-07-12
3,78afa995795e4d85b5d9ceeca43f5fef,F,75.0,100000.0,2017-05-09
4,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,NaN,2017-08-04


In [30]:
transcript.head()

,customer_id,time,event,amount,reward,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,offer_received,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,offer_received,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,offer_received,NaN,NaN,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,offer_received,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,offer_received,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0


In [31]:
# First, let's check the customer ids and offer ids are perfect intersections
set(profile['customer_id']).symmetric_difference(set(transcript['customer_id']))

set()

In [32]:
set(portfolio['offer_id']).symmetric_difference(set(transcript['offer_id']))

{nan}

In [33]:
transcript.offer_id.isna().mean()

0.45330371182315826

In [34]:
# as the ids are the same between sets (except for the nan in the offer_id)
# we can create a key:numbering from the portfolio and the profile and use that
# for the transcript data.


In [35]:
def reconciliate_ids(primary_df, secondary_df, id_column_name):
    """
    Takes hashed ids from two dataframes, and changes them to numbers.
    
    The primary dataframe should be either portfolio or profile.
    The secondary dataframe should be the transcript dataframe.
    
    Input:
    primary_df - either portfolio or profile dataframe
    secondary_df - the transcript dataframe
    id_column_name - a str representing the column you want to reconciliate
    
    Output:
    df1, df2 - dataframes with new id numbers, of the primary and secondary dfs respectively.
    """
    # Renaming for convenience
    df1 = primary_df.copy()
    df2 = secondary_df.copy()
    
    # Get unique ids from primary df
    unique_ids = df1[id_column_name].unique()
    # Create a dict of sequential ids
    id_dict = dict(zip(unique_ids, range(1, len(unique_ids) + 1)))
    # Map primary and secondary dataframe values using the new id dict
    df1[id_column_name] = df1[id_column_name].map(id_dict)
    df2[id_column_name] = df2[id_column_name].map(id_dict)
     
    return df1, df2
    

In [36]:
# Reconciliate the offer id's
portfolio, transcript = reconciliate_ids(portfolio, transcript, 'offer_id')
# Reconciliate the customer id's
profile, transcript = reconciliate_ids(profile, transcript, 'customer_id')

In [37]:
display(portfolio.head())
display(profile.head())
display(transcript.head())

,offer_id,web,email,mobile,social,offer_type,duration,difficulty,reward
0,1,0,1,1,1,bogo,7,10,10
1,2,1,1,1,1,bogo,5,10,10
2,3,1,1,1,0,informational,4,0,0
3,4,1,1,1,0,bogo,7,5,5
4,5,1,1,0,0,discount,10,20,5


,customer_id,gender,age,income,became_member_on
0,1,NaN,NaN,NaN,2017-02-12
1,2,F,55.0,112000.0,2017-07-15
2,3,NaN,NaN,NaN,2018-07-12
3,4,F,75.0,100000.0,2017-05-09
4,5,NaN,NaN,NaN,2017-08-04


,customer_id,time,event,amount,reward,offer_id
0,4,0,offer_received,NaN,NaN,4.0
1,5,0,offer_received,NaN,NaN,5.0
2,6,0,offer_received,NaN,NaN,10.0
3,7,0,offer_received,NaN,NaN,7.0
4,8,0,offer_received,NaN,NaN,2.0


These dataframes are pretty satisfactory. They will be run in a separate python script so the cleaning can be part of a pipeline.